In [1]:
import json
import re
from collections import defaultdict
import numpy as np
import sympy as sy

In [2]:

def parse_side(side_str):
    species = {}
    gas = ''
    for term in side_str.split('+'):
        term = term.strip()
        
        m = re.match(r'(\d*)\s*([A-Za-z0-9_]+)', term)
        if not m:
            raise ValueError(f"Cannot Parse '{side_str}'")
        
        name = m.group(2)
        name_split = name.split("_")
        if len(name_split) == 1:
            gas = name
        
        elif len(name_split) == 2:
            coeff = int(m.group(1)) if m.group(1) else 1
            species[name] = coeff
            
        else:
            raise ValueError(f"Cannot Parse '{side_str}'. It has more than one '_' ")
        
    return species, gas


def load_reactions(path):
    with open(path) as f:
        raw = json.load(f)
    
    reactions_list = []
    all_species = set()
    for r in raw:
        left, right = r['equation'].split('<->') if '<->' in r['equation'] else r['equation'].split('->')
        rev = '<->' in r['equation']
        left, gas  = parse_side(left)
        right, _ = parse_side(right)
        
        all_species |= left.keys() | right.keys()
        reactions_list.append({
            'id':         r['id'],
            'left':     left,
            'right':    right,
            'rate':     r['rate'],
            'gas_specie':      gas,
            'model_dict': r['model_dict']
        })
    species_list = sorted(all_species)
    return reactions_list, species_list


In [3]:
reactions_list, species_list = load_reactions("reactionsSimple.json")

print(reactions_list)
print(species_list)

[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 30.0, 'nu_d': 1000000000000000.0}}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 15.0}}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 6, 'left': {'O_S': 1}, 'right': {'V_S': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 15.0}}, {'id': 7, 'left': {'O_F': 1, 'O_S': 1}, 'right': {'V_F': 1, 'V_S': 1}, 'rate': 'recomb_LH', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 15.0}}]
['O_F', 'O_S', 'V_F', 'V

In [4]:

#### symboolic terms and expressions and conservation laws

groups_species_split = [ele.split('_') for ele in species_list]
group_species = list(set([ele[1] for ele in groups_species_split]))

conservation_laws_dict = {}
for group in group_species:
    vec_species = []
    for specie in groups_species_split:
        if group == specie[1] and specie[0] != "V":
            vec_species.append("_".join(specie))
    
    if len(vec_species) == 0:
        raise ValueError(f"The species _{group} don't have the vacancy state V_{group}. Please modify the reactions list")
    
    conservation_laws_dict["V_" + group] = vec_species


print(group_species)
print(conservation_laws_dict)

rates_list = ["r_"+str(r["id"]) for r in reactions_list]

def make_symbols(name_list):
    syms = sy.symbols(' '.join(name_list))
    return list(syms), dict(zip(name_list, syms))

species_vars, species_map = make_symbols(species_list)
rates_vars, rates_map = make_symbols(rates_list)

#### conservation laws
old_expressions = []
new_expressions = []

for old, new_vec in conservation_laws_dict.items():
    old_expressions.append(species_map[old])
    new_term  = 0.0
    for ele in new_vec:
        new_term = new_term + species_map[ele]
    new_expressions.append(1.0 - new_term)


pairs_laws = list(zip(old_expressions, new_expressions))

print(pairs_laws)
print(rates_map)

['F', 'S']
{'V_F': ['O_F'], 'V_S': ['O_S']}
[(V_F, 1.0 - O_F), (V_S, 1.0 - O_S)]
{'r_1': r_1, 'r_2': r_2, 'r_3': r_3, 'r_4': r_4, 'r_5': r_5, 'r_6': r_6, 'r_7': r_7}


In [5]:
### write the system of equations
S0 = 0.1
F0 = 0.2
ratio_S_F = S0/F0


laws_values_list = list(conservation_laws_dict.values())
eff_species_list = [item for sublist in laws_values_list for item in sublist]

print(eff_species_list)


['O_F', 'O_S']


In [6]:

### write the system of equations
S0 = 0.1
F0 = 0.2
ratio_S_F = S0/F0


laws_values_list = list(conservation_laws_dict.values())
eff_species_list = [item for sublist in laws_values_list for item in sublist]

equations_sym = []

for specie in eff_species_list:

    F_flag = specie.split('_')[1] == "F"
    
    line = []
    for idx, eq in enumerate(reactions_list):
        
        S_flag = False
        double_flag = False
        
        if specie in eq["left"].keys():
            
            rate = "r_" + str(eq["id"])
            term = -1.0
            
            for ele, nu in eq["left"].items():
                term = term * species_map[ele] ** nu
                
                if nu == 2:
                    double_flag = True
                
                if F_flag and not S_flag:
                    S_flag = ele.split('_')[1] == "S"
                
            term = term * rates_map[rate]
            
            if F_flag and S_flag:
                term = term * ratio_S_F
            if double_flag:
                term = term * 2.0
            
            line.append(term)
        
        elif specie in eq["right"].keys():
            
            rate = "r_" + str(eq["id"])
            term = 1.0
            for ele, nu in eq["left"].items():
                term = term * species_map[ele] ** nu
            
                if nu == 2:
                    double_flag = True
            
                if F_flag and not S_flag:
                    S_flag = ele.split('_')[1] == "S"
            
            term = term * rates_map[rate]
            
            if F_flag and S_flag:
                term = term * ratio_S_F
            if double_flag:
                term = term * 2.0
            
            line.append(term)
        
        else:
            pass 
        
    equations_sym.append(sum(line))


equations_sym_new = [equation.subs(pairs_laws) for equation in equations_sym]

#### debugging
for i in range(len(eff_species_list)):
    print(f"d{eff_species_list[i]}/dt = {equations_sym_new[i]}")



dO_F/dt = -0.5*O_F*O_S*r_7 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.5*O_F*r_5*(1.0 - O_S) + 1.0*r_1*(1.0 - O_F)
dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(1.0 - O_S) - 1.0*O_S*r_6 + 1.0*r_3*(1.0 - O_S)


In [7]:

### lambdify expression

eff_species_vars = [species_map[ele] for ele in eff_species_list]

v_fun = sy.lambdify((eff_species_vars, rates_vars), equations_sym_new, modules='numpy')


state_data = [0.3, 0.5]
reaction_data = [0.5]*7

print(v_fun(tuple(state_data), tuple(reaction_data)))

[-0.025000000000000022, 0.0]


In [8]:

### check flux
flux_list_aux = [reaction['gas_specie'] for reaction in  reactions_list]
flux_list = list(set([item for line in flux_list_aux for item in line]))
print(flux_list)

# for reaction in reactions_list:
#     print(reaction['gas'])


['O']


In [9]:

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

exp_dict = {
        'O': 1e3,
        'Tnw': 200,
        'Tw': 200
}


flux_species_list_aux = [reaction['gas_specie'] for reaction in  reactions_list]
flux_species_list = list(set([item for line in flux_species_list_aux for item in line]))
print(flux_species_list)


### flux calculation
for ele in flux_species_list:
        exp_dict[ele] = 2.0*exp_dict[ele]

print(exp_dict)

def adsorption(const_dict, exp_dict, model_dict):
        F0, S0, R = const_dict["F0"], const_dict["S0"], const_dict["R"]
        Tw = exp_dict['Tw']
        SF, E = model_dict['SF'], model_dict['E']
        gas_specie = model_dict['gas_specie']
        
        surface = F0 + S0
        flux  = exp_dict[gas_specie]
        return SF * flux / surface * np.exp(-E / (R * Tw))


print(reactions_list)

model_dict = reactions_list[0]['model_dict']
gas_specie = reactions_list[0]['gas_specie']
model_dict['gas_specie'] = gas_specie

print(model_dict)

rate = adsorption(const_dict, exp_dict, model_dict)
print(rate)


['O']
{'O': 2000.0, 'Tnw': 200, 'Tw': 200}
[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 30.0, 'nu_d': 1000000000000000.0}}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 15.0}}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 0.0}}, {'id': 6, 'left': {'O_S': 1}, 'right': {'V_S': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 15.0}}, {'id': 7, 'left': {'O_F': 1, 'O_S': 1}, 'right': {'V_F': 1, 'V_S': 1}, 'rate': 'recomb_LH', 'gas_specie': '', 'model_dict': {'SF':

In [10]:
#### reaction rates


# flux_list = [2.0 * exp_dict[ele] for ele in flux_species_list]



# print(flux_list)




# def adsorption(const_dict, exp_dict, model_dict, gas):
#     F0, S0, R = const_dict["F0"], const_dict["S0"], const_dict["R"]
#     Tw = exp_dict['Tw']
#     SF, E = model_dict['SF'], model_dict['E']


# #### adsorption
# def adsorption(SF, Flux, T, Ea):
#     surface = const_dict["F0"] + const_dict["S0"]
#     R = const_dict["R"]
#     return SF * Flux / surface * np.exp(-Ea / (R * T))

# #### desorption
# def desorption(SF, nu, T, Ed):
#     R = const_dict["R"]
#     return SF * nu * np.exp(-Ed / (R * T))

# #### create_meta
# def create_meta(SF, Flux, ratio_creation, ratio_temp, T, Ea):
#     surface = const_dict["F0"] + const_dict["S0"]
#     R = const_dict["R"]
#     return SF * Flux / surface * ratio_creation * ratio_temp * np.exp(-Ea / (R * T))

# def destroy_meta(SF, Flux, ratio_destruction, T, Ea):
#     surface = const_dict["F0"] + const_dict["S0"]
#     R = const_dict["R"]
#     return SF * Flux / surface * ratio_destruction * np.exp(-Ea / (R * T))

# def recomb_meta(SF, Flux, ratio_destruction, T, Ea):
#     surface = const_dict["F0"] + const_dict["S0"]
#     R = const_dict["R"]
#     return SF * Flux / surface * (1.0 - ratio_destruction) * np.exp(-Ea / (R * T))

# #### diffusion
# def diffusion(SF, nu, factor, T, Edi):
#     R = const_dict["R"]
#     return SF * nu * np.exp(-Edi / (R * T))

# #### recomb_LH
# def recomb_LH(SF, nu, ratio, T, ED, Ea):
#     R = const_dict["R"]
#     return SF * ratio * nu * np.exp(- (ED + Ea)/(R * T))